In [ ]:
%matplotlib notebook

import SimpleITK as sitk
#import itkwidgets
import os, glob, re, math, sys

In [ ]:
import filebrowser

f=filebrowser.FileBrowser()
print("Select an image from a series")
f.widget()

In [ ]:
# strip off the last 7 characters (###.tif) and replace with *.tif
search_string = f.path[:-7]+'*.tif'
search_string

In [ ]:
import load_slices
slices = load_slices.load_slices(search_string)

In [ ]:
import gui
v = sitk.JoinSeries(slices)

acquire_centers = gui.PointDataAquisition(v)

In [ ]:
import itkwidgets
itkwidgets.view(v, mode='z', cmap='Grayscale')

In [ ]:
import segment

bg = segment.segmentSeriesBackgrounds(slices)

v = sitk.JoinSeries(bg) * 255
itkwidgets.view(v, mode='z', cmap='Grayscale')

In [ ]:
seeds=[]
first_z = 0
picked_seeds = acquire_centers.get_point_indexes()
if len(picked_seeds) != 2:
    print("Warning: there should be two seed points")
    if picked_seeds[0][2] != picked_seeds[1][2]:
        print("Warning: the two seeds are on different slices, using the larger of the two.")
else:
    print(picked_seeds)

    # transform the picked seeds to physical space
    first_z = max(picked_seeds[0][2], picked_seeds[1][2])
    first_bg = bg[first_z]
    for seed in picked_seeds:
        seed_tf = first_bg.TransformIndexToPhysicalPoint(seed[0:2])
        seeds.append(seed_tf)
    print(seeds)

In [ ]:
# Locations of the two holes in physical space
# These seeds sets are used if the interactive seed picking above didn't happen.

# seeds for initial test data set: /Slice & View 01-31/Images/SEM Image
if len(seeds) == 0:
    seeds=[[962,1641],[5293,1660]]

# seeds for 2/18 data set: Slice & View 02-18/Images/SEM Multi-Detector Image
if len(seeds) == 0:
    seeds = [ [678, 216], [618,3456]]

In [ ]:
import importlib
import track_hole
importlib.reload(track_hole)

radius = 500

all_ellipses = []
snum=0
for seed in seeds:
    seed_ellipses = track_hole.track_hole(seed, radius, bg, first_z)
    all_ellipses.extend(seed_ellipses)
#print(all_ellipses)

In [ ]:
import numpy as np
def ellipses2pointset(elist):
    """ convert ellipse centers to a numpy array """
    centers = []
    for e in elist:
        c = e[0]
        centers.append(c)
    np_points = np.array(centers)
    return np_points

In [ ]:
pts = ellipses2pointset(all_ellipses)
#print(pts)

In [ ]:
itkwidgets.view(v, mode='z', cmap='Grayscale',point_sets=[pts])

In [ ]:
import ipywidgets as widgets
w = widgets.Text(
    value='',
    description='Output file'
)
display(w)

In [ ]:
print (w.value)

In [ ]:
#import importlib
import output
#importlib.reload(output)
output.write_results(all_ellipses, w.value)